In [1]:
BATCH_SIZE = 4
EDGE_CROP = 16
NB_EPOCHS = 5
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
# downsampling inside the network
NET_SCALING = None
# downsampling in preprocessing
IMG_SCALING = (1, 1)
# number of validation images to use
VALID_IMG_COUNT = 400
# maximum number of steps_per_epoch in training
MAX_TRAIN_STEPS = 200
AUGMENT_BRIGHTNESS = False

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.io import imread
import matplotlib.pyplot as plt
import gc; gc.enable() # memory is tight
from sklearn.model_selection import train_test_split
import PIL
import tensorflow as tf
from tensorflow import keras

In [3]:
gpus = tf.config.list_physical_devices("GPU")

In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [5]:
allowed_gpus = [6, 7]
gpus = tf.config.list_physical_devices("GPU")
final_gpu_list = [gpus[x] for x in allowed_gpus]
tf.config.set_visible_devices(final_gpu_list, "GPU")

strategy = tf.distribute.MirroredStrategy()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [6]:
csv_file = pd.read_csv('../../ml-data-training/ship_segmentation_data/train_ship_segmentations_v2.csv')
csv_file = csv_file.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()
image_ids, pixels = csv_file['ImageId'].values.tolist(), csv_file['EncodedPixels'].values.tolist()
csv_file['fixed_inputs'] = csv_file['ImageId'].apply(lambda x: '../../ml-data-training/ship_segmentation_data/train_v2/' + x)
csv_file['mask_paths'] = csv_file['ImageId'].apply(lambda x: '../../ml-data-training/ship_segmentation_data/masks_v1/train/' + x.split('.')[0] + '.' + 'png')

In [7]:
csv_file.head()

,ImageId,EncodedPixels,fixed_inputs,mask_paths
0,00003e153.jpg,[nan],../../ml-data-training/ship_segmentation_data/...,../../ml-data-training/ship_segmentation_data/...
1,0001124c7.jpg,[nan],../../ml-data-training/ship_segmentation_data/...,../../ml-data-training/ship_segmentation_data/...
2,000155de5.jpg,[264661 17 265429 33 266197 33 266965 33 26773...,../../ml-data-training/ship_segmentation_data/...,../../ml-data-training/ship_segmentation_data/...
3,000194a2d.jpg,[360486 1 361252 4 362019 5 362785 8 363552 10...,../../ml-data-training/ship_segmentation_data/...,../../ml-data-training/ship_segmentation_data/...
4,0001b1832.jpg,[nan],../../ml-data-training/ship_segmentation_data/...,../../ml-data-training/ship_segmentation_data/...


In [8]:
def split_datasets(csv_file):
    train, test = train_test_split(csv_file, test_size=0.01, random_state=42)
    train, val = train_test_split(train, test_size=0.01, random_state=42)
    return train, val, test

In [9]:
train, val, test = split_datasets(csv_file)

In [10]:
def read_train_imgs(img, mask, shape):
    img = tf.io.read_file(img)
    mask = tf.io.read_file(mask)
    img = tf.image.decode_jpeg(img, channels=3)
    mask = tf.image.decode_jpeg(mask, channels=1)
    img = tf.image.resize(img, size=shape)
    mask = tf.image.resize(mask, size=shape)
    img = img / 255
    mask = mask / 255
    return img, mask

def get_data(data, shape = (256, 256), shuffle = True, repeat = True, batch = True, batch_size = 32):
    imgs, masks = data['fixed_inputs'].values.tolist(), data['mask_paths'].values.tolist()
    shapes = [shape for x in range(len(imgs))]
    tensor = tf.data.Dataset.from_tensor_slices((imgs, masks, shapes))
    tensor = tensor.cache()
    if repeat:
        tensor = tensor.repeat()
    if shuffle:
        tensor = tensor.shuffle(256 * REPLICAS)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        tensor = tensor.with_options(opt)
    tensor = tensor.map(read_train_imgs)
    if batch:
        tensor = tensor.batch(batch_size * REPLICAS)
    tensor = tensor.prefetch(AUTO)
    return tensor

In [11]:
train_dataset = get_data(train, shape = (64, 64), batch_size=32)
val_dataset = get_data(val, shape = (64, 64), batch_size=32)

In [12]:
# def make_image_gen(in_df, batch_size = BATCH_SIZE):
#     input_imgs = in_df['fixed_inputs'].values.tolist()
#     mask_imgs = in_df['mask_paths'].values.tolist()
#     all_batches = [(input_imgs[x], mask_imgs[x]) for x in range(len(input_imgs))]
#     out_rgb = []
#     out_mask = []
#     while True:
#         np.random.shuffle(all_batches)
#         count = 0
#         for c_img_id, c_masks in all_batches:
#             rgb_path = c_img_id
#             c_img = imread(rgb_path)
#             c_mask = imread(c_masks).reshape(768, 768, 1)
#             if IMG_SCALING is not None:
#                 c_img = c_img[::IMG_SCALING[0], ::IMG_SCALING[1]]
#                 c_mask = c_mask[::IMG_SCALING[0], ::IMG_SCALING[1]]
#             out_rgb += [c_img]
#             out_mask += [c_mask]
#             if len(out_rgb)>=batch_size:
#                 yield np.stack(out_rgb, 0)/255.0, np.stack(out_mask, 0)
#                 out_rgb, out_mask=[], []

In [13]:
# train_gen = make_image_gen(train)
# train_x, train_y = next(train_gen)
# print('x', train_x.shape, train_x.min(), train_x.max())
# print('y', train_y.shape, train_y.min(), train_y.max())

In [14]:
# fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (30, 10))
# batch_rgb = montage_rgb(train_x)
# batch_seg = montage(train_y[:, :, :, 0])
# ax1.imshow(batch_rgb)
# ax1.set_title('Images')
# ax2.imshow(batch_seg)
# ax2.set_title('Segmentations')
# ax3.imshow(mark_boundaries(batch_rgb, 
#                            batch_seg.astype(int)))
# ax3.set_title('Outlined Ships')
# fig.savefig('overview.png')

# Make the Validation Set

In [15]:
# valid_x, valid_y = next(make_image_gen(val, 4))
# print(valid_x.shape, valid_y.shape)

# Augment Data

In [16]:
# from keras.preprocessing.image import ImageDataGenerator

In [17]:
# from keras.preprocessing.image import ImageDataGenerator
# # dg_args = dict(featurewise_center = False, 
# #                   samplewise_center = False,
# #                   rotation_range = 15, 
# #                   width_shift_range = 0.1, 
# #                   height_shift_range = 0.1, 
# #                   shear_range = 0.01,
# #                   zoom_range = [0.9, 1.25],  
# #                   horizontal_flip = True, 
# #                   vertical_flip = True,
# #                   fill_mode = 'reflect',
# #                    data_format = 'channels_last')
# # # brightness can be problematic since it seems to change the labels differently from the images 
# # if AUGMENT_BRIGHTNESS:
# #     dg_args[' brightness_range'] = [0.5, 1.5]
# # image_gen = ImageDataGenerator(**dg_args)

# # if AUGMENT_BRIGHTNESS:
# #     dg_args.pop('brightness_range')
# # label_gen = ImageDataGenerator(**dg_args)

# image_gen = ImageDataGenerator()
# label_gen = ImageDataGenerator()

# def create_aug_gen(in_gen, seed = None):
#     # np.random.seed(seed if seed is not None else np.random.choice(range(9999)))
#     for in_x, in_y in in_gen:
#         # seed = np.random.choice(range(9999))
#         # keep the seeds syncronized otherwise the augmentation to the images is different from the masks
#         g_x = image_gen.flow(255*in_x, 
#                              batch_size = in_x.shape[0], 
#                              seed = seed, 
#                              shuffle=True)
#         g_y = label_gen.flow(in_y, 
#                              batch_size = in_x.shape[0], 
#                              seed = seed, 
#                              shuffle=True)

#         yield next(g_x)/255.0, next(g_y)

In [18]:
# cur_gen = create_aug_gen(train_gen)
# t_x, t_y = next(cur_gen)
# print('x', t_x.shape, t_x.dtype, t_x.min(), t_x.max())
# print('y', t_y.shape, t_y.dtype, t_y.min(), t_y.max())
# # only keep first 9 samples to examine in detail
# t_x = t_x[:9]
# t_y = t_y[:9]
# # fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 10))
# # ax1.imshow(montage_rgb(t_x), cmap='gray')
# # ax1.set_title('images')
# # ax2.imshow(montage(t_y[:, :, :, 0]), cmap='gray_r')
# # ax2.set_title('ships')

In [19]:
gc.collect()

0

# Build a Model
Here we use a slight deviation on the U-Net standard

In [20]:
NET_SCALING

In [21]:
UPSAMPLE_MODE

'SIMPLE'

In [28]:
from keras import models, layers
# Build U-Net model
with strategy.scope():
    def upsample_conv(filters, kernel_size, strides, padding):
        return layers.Conv2DTranspose(filters, kernel_size, strides=strides, padding=padding)
    def upsample_simple(filters, kernel_size, strides, padding):
        return layers.UpSampling2D(strides)

    if UPSAMPLE_MODE=='DECONV':
        upsample=upsample_conv
    else:
        upsample=upsample_simple
        
    # Please change below line shape accordingly 32 x 32 is not working and assuming anything lesser than that will not
    # work as well.
    input_img = layers.Input((128, 128, 3), name = 'RGB_Input')
    pp_in_layer = input_img
    if NET_SCALING is not None:
        pp_in_layer = layers.AvgPool2D(NET_SCALING)(pp_in_layer)
        
    pp_in_layer = layers.GaussianNoise(GAUSSIAN_NOISE)(pp_in_layer)
    pp_in_layer = layers.BatchNormalization()(pp_in_layer)

    c1 = layers.Conv2D(8, (3, 3), activation='relu', padding='same') (pp_in_layer)
    c1 = layers.Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
    p1 = layers.MaxPooling2D((2, 2)) (c1)

    c2 = layers.Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
    c2 = layers.Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
    p2 = layers.MaxPooling2D((2, 2)) (c2)

    c3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
    c3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
    p3 = layers.MaxPooling2D((2, 2)) (c3)

    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
    p4 = layers.MaxPooling2D(pool_size=(2, 2)) (c4)


    c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
    c5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same') (c5)

    u6 = upsample(64, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
    c6 = layers.Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

    u7 = upsample(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
    c7 = layers.Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

    u8 = upsample(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
    c8 = layers.Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

    u9 = upsample(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = layers.concatenate([u9, c1], axis=3)
    c9 = layers.Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
    c9 = layers.Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

    d = layers.Conv2D(1, (1, 1), activation='sigmoid') (c9)
    d = layers.Cropping2D((EDGE_CROP, EDGE_CROP))(d)
    d = layers.ZeroPadding2D((EDGE_CROP, EDGE_CROP))(d)
    if NET_SCALING is not None:
        d = layers.UpSampling2D(NET_SCALING)(d)

    seg_model = models.Model(inputs=[input_img], outputs=[d])
    seg_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 RGB_Input (InputLayer)         [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 gaussian_noise_1 (GaussianNois  (None, 128, 128, 3)  0          ['RGB_Input[0][0]']              
 e)                                                                                               
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 128, 128, 3)  12         ['gaussian_noise_1[0][0]']       
 rmalization)                                                                               

In [34]:
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
with strategy.scope():
    def dice_coef(y_true, y_pred, smooth=1):
        intersection = K.sum(y_true * y_pred, axis=[1,2,3])
        union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
        return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)
    def dice_p_bce(in_gt, in_pred):
        return 1e-3*binary_crossentropy(in_gt, in_pred) - dice_coef(in_gt, in_pred)
    def true_positive_rate(y_true, y_pred):
        return K.sum(K.flatten(y_true)*K.flatten(K.round(y_pred)))/K.sum(y_true)
    seg_model.compile(optimizer=Adam(1e-4, decay=1e-6), loss=keras.losses.BinaryCrossentropy(), metrics=[dice_coef, 'binary_accuracy', true_positive_rate])

In [35]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('seg_model')

checkpoint = ModelCheckpoint(weight_path, monitor='val_dice_coef', verbose=1, 
                             save_best_only=True, mode='max', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, 
                                   patience=3, 
                                   verbose=1, mode='max', epsilon=0.0001, cooldown=2, min_lr=1e-6)
early = EarlyStopping(monitor="val_dice_coef", 
                      mode="max", 
                      patience=15) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early, reduceLROnPlat]


In [36]:
train_dataset = get_data(train, shape = (128, 128), batch_size=128)
val_dataset = get_data(val, shape = (128, 128), batch_size=128, repeat=False, shuffle=False)
with strategy.scope():
    model_hist = seg_model.fit(
            train_dataset,
            steps_per_epoch = len(train) // (128 * REPLICAS),
            epochs = 50,
            verbose = 1,
            validation_data = val_dataset,
        )

Epoch 1/50
737/737 [==============================] - 51s 61ms/step - loss: 0.0174 - dice_coef: 0.8016 - binary_accuracy: 0.9988 - true_positive_rate: 0.0000e+00 - val_loss: 0.0181 - val_dice_coef: 0.7906 - val_binary_accuracy: 0.9988 - val_true_positive_rate: 0.0000e+00
Epoch 2/50
737/737 [==============================] - 65s 89ms/step - loss: 0.0174 - dice_coef: 0.8016 - binary_accuracy: 0.9988 - true_positive_rate: 0.0000e+00 - val_loss: 0.0181 - val_dice_coef: 0.7906 - val_binary_accuracy: 0.9988 - val_true_positive_rate: 0.0000e+00
Epoch 3/50
737/737 [==============================] - 66s 90ms/step - loss: 0.0174 - dice_coef: 0.8016 - binary_accuracy: 0.9988 - true_positive_rate: 0.0000e+00 - val_loss: 0.0181 - val_dice_coef: 0.7906 - val_binary_accuracy: 0.9988 - val_true_positive_rate: 0.0000e+00
Epoch 4/50
737/737 [==============================] - 66s 90ms/step - loss: 0.0174 - dice_coef: 0.8016 - binary_accuracy: 0.9988 - true_positive_rate: 0.0000e+00 - val_loss: 0.0181 - v

KeyboardInterrupt: 

In [168]:
step_count = min(MAX_TRAIN_STEPS, train.shape[0]//BATCH_SIZE)
aug_gen = create_aug_gen(make_image_gen(train))
loss_history = [seg_model.fit_generator(aug_gen, 
                             steps_per_epoch=step_count, 
                             epochs=NB_EPOCHS, 
                             validation_data=(valid_x, valid_y),
                             callbacks=callbacks_list,
                            workers=1 # the generator is not very thread safe
                                       )]

/tmp/ipykernel_355982/3169141231.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  loss_history = [seg_model.fit_generator(aug_gen,


Epoch 1/5
 21/200 [==>...........................] - ETA: 10:51 - loss: -0.0921 - dice_coef: 0.0929 - binary_accuracy: 0.6340 - true_positive_rate: nan

KeyboardInterrupt: 

In [ ]:
def show_loss(loss_history):
    epich = np.cumsum(np.concatenate(
        [np.linspace(0.5, 1, len(mh.epoch)) for mh in loss_history]))
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(22, 10))
    _ = ax1.plot(epich,
                 np.concatenate([mh.history['loss'] for mh in loss_history]),
                 'b-',
                 epich, np.concatenate(
            [mh.history['val_loss'] for mh in loss_history]), 'r-')
    ax1.legend(['Training', 'Validation'])
    ax1.set_title('Loss')

    _ = ax2.plot(epich, np.concatenate(
        [mh.history['true_positive_rate'] for mh in loss_history]), 'b-',
                     epich, np.concatenate(
            [mh.history['val_true_positive_rate'] for mh in loss_history]),
                     'r-')
    ax2.legend(['Training', 'Validation'])
    ax2.set_title('True Positive Rate\n(Positive Accuracy)')
    
    _ = ax3.plot(epich, np.concatenate(
        [mh.history['binary_accuracy'] for mh in loss_history]), 'b-',
                     epich, np.concatenate(
            [mh.history['val_binary_accuracy'] for mh in loss_history]),
                     'r-')
    ax3.legend(['Training', 'Validation'])
    ax3.set_title('Binary Accuracy (%)')
    
    _ = ax4.plot(epich, np.concatenate(
        [mh.history['dice_coef'] for mh in loss_history]), 'b-',
                     epich, np.concatenate(
            [mh.history['val_dice_coef'] for mh in loss_history]),
                     'r-')
    ax4.legend(['Training', 'Validation'])
    ax4.set_title('DICE')

show_loss(loss_history)

In [ ]:
seg_model.load_weights(weight_path)
seg_model.save('seg_model.h5')

In [ ]:
pred_y = seg_model.predict(valid_x)
print(pred_y.shape, pred_y.min(), pred_y.max(), pred_y.mean())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (10, 10))
ax.hist(pred_y.ravel(), np.linspace(0, 1, 10))
ax.set_xlim(0, 1)
ax.set_yscale('log', nonposy='clip')

# Prepare Full Resolution Model
Here we account for the scaling so everything can happen in the model itself

In [ ]:
if IMG_SCALING is not None:
    fullres_model = models.Sequential()
    fullres_model.add(layers.AvgPool2D(IMG_SCALING, input_shape = (None, None, 3)))
    fullres_model.add(seg_model)
    fullres_model.add(layers.UpSampling2D(IMG_SCALING))
else:
    fullres_model = seg_model
fullres_model.save('fullres_model.h5')

# Run the test data

In [ ]:
test_paths = os.listdir(test_image_dir)
print(len(test_paths), 'test images found')

In [ ]:
fig, m_axs = plt.subplots(20, 2, figsize = (10, 40))
[c_ax.axis('off') for c_ax in m_axs.flatten()]
for (ax1, ax2), c_img_name in zip(m_axs, test_paths):
    c_path = os.path.join(test_image_dir, c_img_name)
    c_img = imread(c_path)
    first_img = np.expand_dims(c_img, 0)/255.0
    first_seg = fullres_model.predict(first_img)
    ax1.imshow(first_img[0])
    ax1.set_title('Image')
    ax2.imshow(first_seg[0, :, :, 0], vmin = 0, vmax = 1)
    ax2.set_title('Prediction')
fig.savefig('test_predictions.png')

# Submission
Since gneerating the submission takes a long time and quite a bit of memory we run it in a seperate kernel located at https://www.kaggle.com/kmader/from-trained-u-net-to-submission-part-2 
That kernel takes the model saved in this kernel and applies it to all the test data